In [10]:
import pandas as pd
import seaborn as sea
import numpy as np
import os

import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
import plotly.figure_factory as ff
from plotly.graph_objs import *
import warnings

In [11]:
plotly_key = os.environ["PLOTLY_KEY"]
mapbox_token = "pk.eyJ1IjoiYW14bW9uZ29vc2UiLCJhIjoiY2plbG1iYml3MXdrYjJxbW80YW53dWE0bCJ9.3VkhNFfkrJISw42FzdDbRA"
plotly.tools.set_credentials_file(username='amxmongoose', api_key=plotly_key)

Data Used:
    1. U.S. census data
    2. County health data
    3. Air quality data
    
Objectives:
    1. Identify relationship between geographic information and health statistics
    2. Determine is there is a correlation between geographic air quality and health quality
    3. Show healthiest/unhealthiest places in the united states both in air quality and health statistics
    

In [45]:
fips_path = os.path.join('Data Sets', 'US_FIPS_Codes 2.csv')
fields = ['State', 'County Name', 'FIPS State', 'FIPS County']
fips_df = pd.read_csv(fips_path, skipinitialspace=True, usecols=fields)
fips_df = fips_df.rename(columns={'County Name':'County'})
fips_df['FIPS State'] = fips_df['FIPS State'].apply(lambda x: '{0:0>2}'.format(int(x)))
fips_df['FIPS County'] = fips_df['FIPS County'].apply(lambda x: '{0:0>3}'.format(int(x)))
fips_df["Fips"] = fips_df["FIPS State"].map(str) + fips_df["FIPS County"]

In [46]:
health_data_path = os.path.join('Data Sets', 'health_data', 'SUMMARYMEASURESOFHEALTH.csv')
fields = ['State_FIPS_Code', 'County_FIPS_Code','CHSI_County_Name','CHSI_State_Name','ALE','All_Death','Health_Status','CHSI_State_Abbr']
health_df = pd.read_csv(health_data_path, skipinitialspace=True, usecols=fields)
health_df = health_df.rename(columns={'CHSI_County_Name':'County','CHSI_State_Name':'State','CHSI_State_Abbr':'Abbr'})

In [47]:
census_data_path = os.path.join('Data Sets', 'acs2015_county_data.csv')
fields = ['State', 'County', 'TotalPop', 'Income', 'IncomePerCap', 'Poverty',
       'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Employed', 'Unemployment']
census_df = pd.read_csv(census_data_path, skipinitialspace=True, usecols=fields)

In [48]:
air_data_path = os.path.join('Data Sets', 'annual_aqi_by_county_2017.csv')
fields = ['State', 'County', 'Max AQI', 'Median AQI','Unhealthy Days']
air_df = pd.read_csv(air_data_path, skipinitialspace=True, usecols=fields)

In [49]:
merged_df = pd.merge(health_df,census_df,on=['County','State'],how='inner')

In [50]:
merged_df_1 = pd.merge(fips_df,census_df,on=['County','State'],how='inner')
merged_df_full = pd.merge(merged_df_1,health_df,on=['County','State'],how='outer')
merged_df_full = pd.merge(merged_df_full,air_df,on=['County','State'],how='outer')

In [51]:
merged_df = pd.merge(merged_df,air_df,on=['County','State'],how='outer')
merged_df = merged_df.dropna(subset = ['Max AQI','Median AQI','State_FIPS_Code','County_FIPS_Code'])
merged_df['County_FIPS_Code'] = merged_df['County_FIPS_Code'].apply(lambda x: '{0:0>3}'.format(int(x)))
merged_df['State_FIPS_Code'] = merged_df['State_FIPS_Code'].apply(lambda x: '{0:0>2}'.format(int(x)))
merged_df['Fips'] = merged_df['State_FIPS_Code'].map(str) + merged_df['County_FIPS_Code']
merged_df.head()

,State_FIPS_Code,County_FIPS_Code,County,State,Abbr,ALE,All_Death,Health_Status,TotalPop,Income,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,Unemployment,Unhealthy Days,Max AQI,Median AQI,Fips
1,01,003,Baldwin,Alabama,AL,76.6,856.9,15.4,195121.0,50254.0,...,1.0,1.4,3.9,26.4,85953.0,7.5,0.0,108.0,41.0,01003
13,01,027,Clay,Alabama,AL,74.9,950.3,-1111.1,13537.0,35327.0,...,1.8,0.9,2.1,30.3,5180.0,9.4,0.0,66.0,27.0,01027
16,01,033,Colbert,Alabama,AL,75.3,964.4,20.3,54444.0,40576.0,...,0.9,0.7,1.5,22.8,21290.0,9.0,0.0,63.0,41.0,01033
24,01,049,DeKalb,Alabama,AL,75.1,960.6,22.0,71068.0,38192.0,...,0.6,0.9,2.1,22.8,27778.0,7.7,0.0,80.0,40.0,01049
25,01,051,Elmore,Alabama,AL,75.2,946.9,15.1,80763.0,53555.0,...,0.4,1.2,2.2,28.5,33636.0,8.3,0.0,58.0,38.0,01051


In [40]:
station_path = os.path.join('Data Sets', 'epa_air_quality_annual_summary 2.csv')
fields = ['latitude', 'longitude', 'arithmetic_mean', 'year','state_name', 'city_name', 'observation_percent','observation_count']
station_df = pd.read_csv(station_path, usecols=fields)
station_df = station_df.loc[station_df['year'] == 2015]
station_df = station_df.reset_index(drop=True)
station_df.head()

,latitude,longitude,year,observation_count,observation_percent,arithmetic_mean,state_name,city_name
0,42.281869,-83.151415,2015,53,88,1.061132,Michigan,Detroit
1,47.597222,-122.319722,2015,51,85,2.642667,Washington,Seattle
2,39.922867,-75.186921,2015,57,95,0.527228,Pennsylvania,Philadelphia
3,40.786878,-81.394186,2015,56,93,0.002786,Ohio,Canton
4,32.668987,-94.167457,2015,57,95,1.031579,Texas,NaN


In [96]:
latitudes = station_df['latitude']
longitudes = station_df['longitude']
text = station_df['observation_count']
total_obs = station_df['observation_count'].sum()
obs_loc = len(station_df['observation_count'])
avg_obs = total_obs/obs_loc

scl = [[0, 'rgb(150,0,90)'],
       [0.01, 'rgb(0, 0, 200)'],
       [0.05,'rgb(0, 25, 255)'],
       [0.075,'rgb(0, 152, 255)'],
       [0.1,'rgb(44, 255, 150)'],
       [0.25,'rgb(151, 255, 0)'],
       [0.5,'rgb(255, 234, 0)'],
       [1.0,'rgb(255, 111, 0)'],
       [1.5,'rgb(255, 0, 0)']];


data = Data([
    Scattermapbox(
        lat=latitudes,
        lon=longitudes,
        mode='markers',
        marker= Marker(
            color=(station_df['observation_count']/avg_obs),
            colorscale = scl,
            cmin = 0,
            cmax = 1,
            reversescale = True,
            opacity = 0.4,
            size = 10,
            showscale=True,
         ),
         text=text,
     )
 ])
      

layout = Layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_token,
        bearing=0,
        center=dict(
            lat=39,
            lon=-98
        ),
        pitch=0,
        zoom=3,
        style='light'
    ),
)

fig = dict(data=data, layout=layout)
py.iplot(fig)

PlotlyRequestError: Hi there, you've reached the threshold of 100 combined image exports and chart saves per day. If you need to raise your daily limit, consider upgrading to a Student or Personal Plan (see: https://plot.ly/products/cloud).

In [67]:
latitudes = station_df['latitude']
longitudes = station_df['longitude']
text = station_df['observation_count']


data = Data([
    Scattermapbox(
        lat=latitudes,
        lon=longitudes,
        mode='markers',
        marker=Marker(
             size=8,
             color=station_df['observation_count'],
             colorscale= 'Viridis',
             showscale=True,
             cauto=True,
             opacity = 0.5,
         ),
         text=text,
     )
 ])

layout = Layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_token,
        bearing=0,
        center=dict(
            lat=39,
            lon=-98
        ),
        pitch=0,
        zoom=3,
        style='light'
    ),
)

fig = dict(data=data, layout=layout)
py.iplot(fig)

In [31]:
warnings.filterwarnings('ignore')

fips = merged_df_1['Fips']

values_max = merged_df_1['TotalPop']
bins_max = [0,5000,10000,25000,50000,75000,100000,250000,500000,1000000,2500000,5000000]

fig1 = ff.create_choropleth(fips=fips, values=values_max, binning_endpoints=bins_max)

py.iplot(fig1)

The draw time for this plot will be slow for clients without much RAM.


In [27]:
fips = merged_df['Fips']

values_air = merged_df['Max AQI']
bins_air = [0,10,25,50,75,100,125,150,175,200,250,500,1000]
fig2 = ff.create_choropleth(fips=fips, values=values_air, binning_endpoints=bins_air)
py.iplot(fig2)